Generate envelope per floor

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import random
from grid import *


In [3]:
def gen_rect_int(size):
    """
    Function to generate a rectangle and placing accurate corner pieces as well

    """
    x, y = size
    rect = []
    
    # Using integers for walls: 6, 10, 12, 5, 3, 9
    top = [1] + (x-2)*[1] + [1]
    mid = [1] + [17] * (x - 2) + [1]
    bottom = [1] + [1] * (x - 2) + [1]
    
    # Construct the rectangle
    rect = np.array([top] + [mid] * (y - 2) + [bottom])
    
    return rect

def glue_rect_int(rect_a, rect_b, offset=0, plot=True):
    """
    Glue two rectangles (rect_a and rect_b) together with a specified offset.
    Handles the corners and connecting walls, represented with integers.
    """
    y_a, x_a = rect_a.shape
    y_b, x_b = rect_b.shape
    print(f"x_a={x_a}, y_a={y_a}")
    print(f"x_b={x_b}, y_b={y_b}")
    print(f"Offset: {offset}")
    
    # Create combined shape (rows, columns)
    combined_shape = (y_a + y_b - 1, max(x_a, offset + x_b))
    y_c, x_c = combined_shape
    combined_rect = np.full(combined_shape, 0, dtype=int)  # Use 0 to represent empty spaces

    # Place rect_a into the combined grid
    combined_rect[:y_a, :x_a] = rect_a.copy()
    # Place rect_b with offset into the combined grid
    combined_rect[y_a-1 : y_a + y_b - 1, offset : offset + x_b] = rect_b.copy()

    # Fix corners and destroy connecting wall
    for j in range(x_b):
        # Left-hand side corner
        if j == 0:
            combined_rect[y_a-1, offset] = 12 if offset > 0 else 5

        # Destroy wall in between
        if 0 < j < x_a - 1 - offset:
            combined_rect[y_a-1, offset + j] = 17  # Empty inside space

        # Right-hand side corner
        if j == x_a - 1 - offset:
            if offset + x_b < x_a:
                combined_rect[y_a-1, offset+j] = 6
            elif offset + x_b == x_a:
                combined_rect[y_a-1, offset+j] = 5
            else:
                combined_rect[y_a-1, offset+j] = 3

    return combined_rect

In [ ]:
def gen_rect_int(size):
    """
    Function to generate a rectangle and placing accurate corner pieces as well
    """
    x, y = size
    rect = []
    
    # Using integers for walls: 6, 10, 12, 5, 3, 9
    top = [1] + (x-2)*[1] + [1]
    mid = [1] + [255] * (x - 2) + [1]
    bottom = [1] + [1] * (x - 2) + [1]
    
    # Construct the rectangle
    rect = np.array([top] + [mid] * (y - 2) + [bottom])
    
    return rect

def glue_rect_int(rect_a, rect_b, offset=0, plot=True):
    """
    Glue two rectangles (rect_a and rect_b) together with a specified offset.
    Handles the corners and connecting walls, represented with integers.
    """
    y_a, x_a = rect_a.shape
    y_b, x_b = rect_b.shape
    print(f"x_a={x_a}, y_a={y_a}")
    print(f"x_b={x_b}, y_b={y_b}")
    print(f"Offset: {offset}")
    
    # Create combined shape (rows, columns)
    combined_shape = (y_a + y_b - 1, max(x_a, offset + x_b))
    y_c, x_c = combined_shape
    combined_rect = np.full(combined_shape, 0, dtype=int)  # Use 0 to represent empty spaces

    # Place rect into the combined grid
    combined_rect[:y_a, :x_a] = rect_a.copy()
    combined_rect[y_a-1 : y_a + y_b - 1, offset : offset + x_b] = rect_b.copy()

    # Fix corners and destroy connecting wall
    for j in range(x_b):
        # Left-hand side corner
        if j == 0:
            combined_rect[y_a-1, offset] = 12 if offset > 0 else 5

        # Destroy wall in between
        if 0 < j < x_a - 1 - offset:
            combined_rect[y_a-1, offset + j] = 17  # Empty inside space

        # Right-hand side corner
        if j == x_a - 1 - offset:
            if offset + x_b < x_a:
                combined_rect[y_a-1, offset+j] = 6
            elif offset + x_b == x_a:
                combined_rect[y_a-1, offset+j] = 5
            else:
                combined_rect[y_a-1, offset+j] = 3

    return combined_rect

class FloorGenerator:
    def __init__(self):
        self.base_floor = self._generate_base_floor()
        self.num_copies = random.randint(1, 8)
        self.floors = [self._create_floor_copy(i) for i in range(self.num_copies)]
        
    def _generate_base_floor(self):
        """Generate a single random floor layout"""
        if random.random() < 0.5:
            # Rectangle floor
            x, y = random.randint(3, 25), random.randint(3, 25)
            layout = gen_rect_int((x, y)),
            #seed1 = random.randint(x, y)
            #seed2 = random.randint(x, y)
            #seed3 = random.randint(x, y)
            #seeds = [(seed1), (seed2), (seed3)]
            return {
                'type': 'rectangle',
                'dimensions': (x, y),
                'layout': layout,
                #'seeds': seeds
            }
        else:
            # L-shaped floor
            x1, y1 = random.randint(3, 25), random.randint(3, 25)
            x2, y2 = random.randint(3, 25), random.randint(3, 25)
            offset = random.randint(0, x1 - 1)
            
            rect1 = gen_rect_int((x1, y1))
            rect2 = gen_rect_int((x2, y2))
            layout = glue_rect_int(rect1, rect2, offset)
            #seed1 = random.randint(x1, y1)
            #seed2 = random.randint(x1, y1)
            #seed3 = random.randint(x1, y1)
            #seed4 = random.randint(x2+offset, y2+offset)
            #seeds = [(seed1), (seed2), (seed3), (seed4)]
            return {
                'type': 'L-shape',
                'components': {
                    'base': (x1, y1),
                    'extension': (x2, y2),
                    'offset': offset
                },
                'layout': layout,
                #'seeds': seeds
            }

    def _create_floor_copy(self, index):
        """Create a unique copy of the base floor with index"""
        floor_copy = copy.deepcopy(self.base_floor)
        floor_copy['copy_number'] = index + 1
        return floor_copy

    def get_all_floors(self):
        """Return all floor copies with metadata"""
        return self.floors

    def display_floor(self, index):
        """Show detailed information about a specific floor copy"""
        floor = self.floors[index]
        print(f"\nFloor Copy #{floor['copy_number']}")
        print(f"Original Type: {floor['type'].upper()}")
        
        if floor['type'] == 'rectangle':
            print(f"Dimensions: {floor['dimensions']}")
        else:
            print(f"Components:")
            print(f"- Base: {floor['components']['base']}")
            print(f"- Extension: {floor['components']['extension']}")
            print(f"- Offset: {floor['components']['offset']}")
            
        print("Layout:")
        print(floor['layout'])

    def display_all(self):
        """Display all floor copies"""
        print(f"Total floor copies: {self.num_copies}")
        for floor in self.floors:
            self.display_floor(floor['copy_number'] - 1)

In [22]:
def generate_random_floorplan():
    if random.random() < 0.5:
        x = random.randint(3, 25)
        y = random.randint(3, 25)
        return gen_rect_int((x, y))
    else:
        x1 = random.randint(3, 25)
        y1 = random.randint(3, 25)
        x2 = random.randint(3, 25)
        y2 = random.randint(3, 25)
        rect1 = gen_rect_int((x1, y1))
        rect2 = gen_rect_int((x2, y2))
        max_offset = x1 - 1
        offset = random.randint(0, max_offset)
        combined = glue_rect_int(rect1, rect2, offset=offset, plot=False)
        return combined    

In [26]:
combined = generate_random_floorplan()
print(combined)

x_a=21, y_a=16
x_b=14, y_b=22
Offset: 9
[[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  0  0]
 [ 1 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17  1  0  0]
 [ 1 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17  1  0  0]
 [ 1 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17  1  0  0]
 [ 1 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17  1  0  0]
 [ 1 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17  1  0  0]
 [ 1 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17  1  0  0]
 [ 1 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17  1  0  0]
 [ 1 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17  1  0  0]
 [ 1 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17  1  0  0]
 [ 1 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17  1  0  0]
 [ 1 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17  1  0  0]
 [ 1 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17  1  0  0]
 [ 1 17 17 17 17 17 17 1

In [ ]:
floor_gen = FloorGenerator()
floor_gen.display_all()

# Access individual copies:
first_copy = floor_gen.get_all_floors()[0]
result = region_growing_simultaneous(floor_gen['layout'], floor_gen['seeds'])
print(result.shape)
result_boxed = generate_mapping_rectangles(np.array(result))

plot_floorplan(result_boxed, seed)

print("\nFirst copy layout:")
print(first_copy['layout'])